In [6]:
from abc import ABC, abstractmethod
from typing import Any
from pprint import pprint

# 06 Polymorphism - Demos

## Method inheritance
This practice is called method inheritance and can be useful when subclasses don't require any specific changes in the initialization process.

In [2]:
class Vehicle(ABC):
    def __init__(self, fuel_quantity, fuel_consumption) -> None:
        self.fuel_quantity = fuel_quantity
        self.fuel_consumption = fuel_consumption


class Car(Vehicle):
    pass


toyota = Car(60, 8)

## Name mangling is only for private attrs

### Transformation occurs at compile-time

In Python, when a class attribute is prefixed with double underscores (\_\_), name mangling is applied to the attribute.  
Name mangling transforms the attribute name by adding a prefix consisting of a single underscore (\_) and the class name.  
**This transformation occurs at compile-time**.

In [17]:
class Tile:
    MAX_SIZE = "110 X 110"
    __PRICE_COEF = 1.0

    def print_parent_attrs(self):
        print(self.__PRICE_COEF)


class BathroomTile(Tile):
    MAX_SIZE = "65 X 65"
    __PRICE_COEF = 1.4

    def print_child_attrs(self):
        print(self.__PRICE_COEF)


bianco = BathroomTile()
bianco.print_parent_attrs()  # Accesses _Tile__PRICE_COEF
bianco.print_child_attrs()   # Accesses _BathroomTile__PRICE_COEF

1.0
1.4


### List of names in an object = dir()

The dir() function returns a **sorted list of names in the given object**, including attributes, methods, and special names.

In [3]:
class Tile:
    MAX_SIZE = "110 X 110"
    __PRICE_COEF = 1.0


class BathroomTile(Tile):
    MAX_SIZE = "65 X 65"
    __PRICE_COEF = 1.4

    def __init__(self) -> None:
        self.price = 33.80


def remove_magic_methods(attributes):
    return [attr for attr in attributes if not attr.startswith("__")]


# Print the names in the instance nad in the class.
all_objects = {
    "Tile class": Tile,
    "BathroomTile class": BathroomTile,
    "Instance": BathroomTile(),
}

for name, obj in all_objects.items():
    attributes = remove_magic_methods(dir(obj))
    print(f"{name:<20}{str(attributes)}")

Tile class          ['MAX_SIZE', '_Tile__PRICE_COEF']
BathroomTile class  ['MAX_SIZE', '_BathroomTile__PRICE_COEF', '_Tile__PRICE_COEF']
Instance            ['MAX_SIZE', '_BathroomTile__PRICE_COEF', '_Tile__PRICE_COEF', 'price']


### Dictionary of object's attributes = \_\_dict\_\_

A dictionary or other mapping object used to store an object's (writable) attributes.  
var() is equivalent to object.\_\_dict\_\_

In [12]:
for name, obj in all_objects.items():
    print(name)  
    pprint(vars(obj))
    print('-'*80)

Tile class
mappingproxy({'MAX_SIZE': '110 X 110',
              '_Tile__PRICE_COEF': 1.0,
              '__dict__': <attribute '__dict__' of 'Tile' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Tile' objects>})
--------------------------------------------------------------------------------
BathroomTile class
mappingproxy({'MAX_SIZE': '65 X 65',
              '_BathroomTile__PRICE_COEF': 1.4,
              '__doc__': None,
              '__init__': <function BathroomTile.__init__ at 0x000001C371103D90>,
              '__module__': '__main__'})
--------------------------------------------------------------------------------
Instance
{'price': 33.8}
--------------------------------------------------------------------------------


## No mangling for non-public attrs

In [15]:
class Tile:
    MAX_SIZE = "110 X 110"
    _PRICE_COEF = 1.0


class BathroomTile(Tile):
    MAX_SIZE = "65 X 65"
    _PRICE_COEF = 1.4

    def __init__(self) -> None:
        self.price = 33.80


def remove_magic_methods(attributes):
    return [attr for attr in attributes if not attr.startswith("__")]


# Print the names in the instance nad in the class.
all_objects = {
    "Tile class": Tile,
    "BathroomTile class": BathroomTile,
    "Instance": BathroomTile(),
}

for name, obj in all_objects.items():
    attributes = remove_magic_methods(dir(obj))
    print(f"{name:<20}{str(attributes)}")

Tile class          ['MAX_SIZE', '_PRICE_COEF']
BathroomTile class  ['MAX_SIZE', '_PRICE_COEF']
Instance            ['MAX_SIZE', '_PRICE_COEF', 'price']


### Accessing non-public class attr

Можем да зададем нови стойности на поле на класа в наследник ако ползваме non-public (една долна черта) или стандартни (без долни черти в началото) полета.

Ако полето е private, имаме погрозняване. Името на такова поле ще е винаги `_Tile__PRICE_COEF` защото init методът ще се вика винаги от вътрешността на Tile класа.

In [18]:
class Tile:
    MAX_SIZE = "110 X 110"
    _PRICE_COEF = 1.0

    def __init__(self) -> None:
        self.coef = self._PRICE_COEF

class BathroomTile(Tile):
    MAX_SIZE = "65 X 65"
    _PRICE_COEF = 1.4

    
print(Tile().coef)
print(BathroomTile().coef)

1.0
1.4


In [20]:
class Tile:
    MAX_SIZE = "110 X 110"
    __PRICE_COEF = 1.0

    def __init__(self) -> None:
        self.coef = self.__PRICE_COEF

class BathroomTile(Tile):
    MAX_SIZE = "65 X 65"
    __PRICE_COEF = 1.4

    
print(Tile().coef)
print(BathroomTile().coef)

1.0
1.0


### Good solution

In [ ]:
class InsufficientFuelError(Exception):
    """Custom exception that is raised when the remaining fuel is not enough."""

    def _init_(self, message) -> None:
        super()._init_(message)


class Vehicle:
    _AC_FUEL_CONSUMPTION = 0
    _FUEL_RETENTION_COEF = 1

    def __init__(self, fuel_quantity: float, fuel_consumption: float) -> None:
        """Main consructor.

        Args:
            fuel_quantity (float): how much fuel in liters is currently in the tank
            fuel_consumption (float): in liters/km
        """
        self.fuel_quantity = fuel_quantity
        self.fuel_consumption = fuel_consumption + self._AC_FUEL_CONSUMPTION

    @classmethod
    def from_liters_per_100km(
        cls, fuel_quantity: float, fuel_consumption_per_100km: float
    ):
        """Alternate constructor for fuel consumption in liters / 100km."""
        fuel_consumption = fuel_consumption_per_100km / 100 + cls._AC_FUEL_CONSUMPTION
        return cls(fuel_quantity, fuel_consumption)

    def drive(self, distance):
        consumed_fuel = distance * self.fuel_consumption
        if consumed_fuel > self.fuel_quantity:
            raise InsufficientFuelError("Too little fuel remaining in the tank.")
        self.fuel_quantity -= consumed_fuel

    def refuel(self, quantity):
        self.fuel_quantity += quantity * self._FUEL_RETENTION_COEF


class Car(Vehicle):
    _AC_FUEL_CONSUMPTION = 0.9
    _FUEL_RETENTION_COEF = 1


class Truck(Vehicle):
    _AC_FUEL_CONSUMPTION = 1.6
    _FUEL_RETENTION_COEF = 0.95


car = Car(20, 5)
car.drive(3)
print(car.fuel_quantity)
car.refuel(10)
print(car.fuel_quantity)


truck = Truck(100, 15)
truck.drive(5)
print(truck.fuel_quantity)
truck.refuel(50)
print(truck.fuel_quantity)


In [2]:
from abc import ABC, abstractmethod

# Make an abstrat class Vehicle.
# Make a class Vehicle that is an AbstractBaseClass.


class Vehicle(ABC):
    @abstractmethod
    def drive(self):
        pass

    @abstractmethod
    def refuel(self):
        pass


class Car(Vehicle):
    def drive(self, distance):
        pass

    def refuel(self, fuel):
        pass


class Truck(Vehicle):
    def drive(self, distance):
        pass

    def refuel(self, fuel):
        pass


toyota = Car()
volvo = Truck()

In [2]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name


class Dog(Animal):
    sound = "bark"

    def __init__(self, name) -> None:
        super().__init__(name)

    def make_sound(self) -> None:
        print(self.sound)


class Cat(Animal):
    sound = "meow"

    def __init__(self, name) -> None:
        super().__init__(name)

    def make_sound(self) -> None:
        print(self.sound)


sparky = Dog("Sparky")
tom = Cat("Tom")

sparky.make_sound()
tom.make_sound()

bark
meow


In [8]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    @classmethod
    def make_sound(cls):
        print(cls.sound)


class Dog(Animal):
    sound = "bark"

    def __init__(self, name) -> None:
        super().__init__(name)


class Cat(Animal):
    sound = "meow"

    def __init__(self, name) -> None:
        super().__init__(name)


sparky = Dog("Sparky")
tom = Cat("Tom")

sparky.make_sound()
tom.make_sound()

bark
meow


In [5]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    def make_sound(self):
        print(self.sound)


class Dog(Animal):
    sound = "bark"

    def __init__(self, name) -> None:
        super().__init__(name)


class Cat(Animal):
    sound = "meow"

    def __init__(self, name) -> None:
        super().__init__(name)


sparky = Dog("Sparky")
tom = Cat("Tom")

sparky.make_sound()
tom.make_sound()

bark
meow


In [1]:
class Animal(ABC):
    def __init__(self, name) -> None:
        self.name = name

    @abstractmethod
    def make_sound(self):
        pass


class Dog(Animal):
    sound = "bark"

    def __init__(self, name) -> None:
        super().__init__(name)

    def make_sound(self):
        print(self.sound)


class Cat(Animal):
    sound = "meow"

    def __init__(self, name) -> None:
        super().__init__(name)

    def make_sound(self):
        print(self.sound)


sparky = Dog("Sparky")
tom = Cat("Tom")

sparky.make_sound()
tom.make_sound()

NameError: name 'ABC' is not defined